In [10]:
# to visualize data
import matplotlib.pyplot as plt
# to browse directory
import os
# to manage dataframes
import pandas as pd
import numpy as np
# to calculate the time to run the codes
from datetime import datetime
# the deep learning framework (TensorFlow)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0,DenseNet121,MobileNetV2,EfficientNetV2S
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
SEED_VALUE = 1234
np.random.seed(SEED_VALUE)
import random
random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)
import h5py
import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

: 

In [1]:
NUM_CLASSES = 7
IMG_SHAPE = (224, 224, 3)
BATCH_SIZE = 32
SEED_VALUE = 1234

TRAIN_EPOCH = 100
TRAIN_LR_PATIENCE = 5
TRAIN_MIN_LR = 1e-6

TRAIN_LR = 0.001
TRAIN_ES_PATIENCE = 5
TRAIN_DROPOUT = 0.1

FT_EPOCH = 500
FT_LR = 1e-5
FT_LR_DECAY_STEP = 80.0
FT_LR_DECAY_RATE = 1
FT_ES_PATIENCE = 20
FT_DROPOUT = 0.2

ES_LR_MIN_DELTA = 0.003

In [ ]:
#import os
#import shutil
'''
# Set paths
DATASET_PATH = "C:\\Users\\Matthew\\Downloads\\archive (13)\\UTKFace"
TRAIN_PATH = os.path.join(DATASET_PATH, "train")
VAL_PATH = os.path.join(DATASET_PATH, "val")
TEST_PATH = os.path.join(DATASET_PATH, "test")

# Function to create age subfolders
def create_age_subfolders(base_path):
    for age in range(117):  # Age ranges from 0 to 116
        age_folder = os.path.join(base_path, str(age))
        os.makedirs(age_folder, exist_ok=True)

# Create age subfolders in train, val, and test directories
create_age_subfolders(TRAIN_PATH)
create_age_subfolders(VAL_PATH)
create_age_subfolders(TEST_PATH)

# Function to move files to appropriate age folders
def move_files_to_age_folders(source_path, destination_path):
    all_files = [f for f in os.listdir(source_path) if os.path.isfile(os.path.join(source_path, f))]
    
    for file in all_files:
        try:
            age = int(file.split('_')[0])
            age_folder = os.path.join(destination_path, str(age))
            src = os.path.join(source_path, file)
            dst = os.path.join(age_folder, file)
            shutil.move(src, dst)
        except ValueError:
            print(f"Skipping file with invalid name format: {file}")
        except Exception as e:
            print(f"Error moving file {file}: {str(e)}")

# Move files from original train, val, and test folders to age subfolders
move_files_to_age_folders(os.path.join(DATASET_PATH, "train"), TRAIN_PATH)
move_files_to_age_folders(os.path.join(DATASET_PATH, "val"), VAL_PATH)
move_files_to_age_folders(os.path.join(DATASET_PATH, "test"), TEST_PATH)

print("Files moved to age-specific folders in train, val, and test directories.")'''

'\n# Set paths\nDATASET_PATH = "C:\\Users\\Matthew\\Downloads\\archive (13)\\UTKFace"\nTRAIN_PATH = os.path.join(DATASET_PATH, "train")\nVAL_PATH = os.path.join(DATASET_PATH, "val")\nTEST_PATH = os.path.join(DATASET_PATH, "test")\n\n# Function to create age subfolders\ndef create_age_subfolders(base_path):\n    for age in range(117):  # Age ranges from 0 to 116\n        age_folder = os.path.join(base_path, str(age))\n        os.makedirs(age_folder, exist_ok=True)\n\n# Create age subfolders in train, val, and test directories\ncreate_age_subfolders(TRAIN_PATH)\ncreate_age_subfolders(VAL_PATH)\ncreate_age_subfolders(TEST_PATH)\n\n# Function to move files to appropriate age folders\ndef move_files_to_age_folders(source_path, destination_path):\n    all_files = [f for f in os.listdir(source_path) if os.path.isfile(os.path.join(source_path, f))]\n    \n    for file in all_files:\n        try:\n            age = int(file.split(\'_\')[0])\n            age_folder = os.path.join(destination_pat

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Path dataset
TRAIN_PATH = "C:\\Users\\Matthew\\Downloads\\archive (13)\\UTKFace\\train"
VAL_PATH = "C:\\Users\\Matthew\\Downloads\\archive (13)\\UTKFace\\val"
TEST_PATH = "C:\\Users\\Matthew\\Downloads\\archive (13)\\UTKFace\\test"

# Transformasi untuk preprocessing gambar
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageFolder(root=TRAIN_PATH, transform=transform)
val_dataset = ImageFolder(root=VAL_PATH, transform=transform)
test_dataset = ImageFolder(root=TEST_PATH, transform=transform)

# DataLoader untuk training, validation, dan test set
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm 
class AgeRegressionModel(nn.Module):
    def __init__(self, model_name='volo_d1_224', pretrained=True):
        super(AgeRegressionModel, self).__init__()
        self.backbone = timm.create_model(model_name, pretrained=pretrained)
        self.backbone.head = nn.Sequential(
            nn.Linear(self.backbone.head.in_features, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.backbone(x)

# Inisialisasi model
model = AgeRegressionModel(model_name='volo_d1_224')

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
epochs = 10

for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, targets.unsqueeze(1))  # targets perlu diubah dimensinya
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.float().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            
            val_loss += loss.item() * inputs.size(0)
    
    # Calculate average losses
    train_loss = train_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    
    print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

print('Finished Training')

C:\Users\Matthew\anaconda3\envs\TestENV\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 1000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Evaluate on test set
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.float().to(device)
        
        outputs = model(inputs)
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(targets.cpu().numpy())

predictions = np.array(predictions)
actuals = np.array(actuals)

# Calculate metrics
r2 = r2_score(actuals, predictions)
mae = mean_absolute_error(actuals, predictions)
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)

print(f'R²: {r2:.4f}')
print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MSE: {mse:.4f}')

In [ ]:
#pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#pip install timm

In [ ]:
# KLIK 'Q' NANTI KALAU MAU QUIT CV2
cap = cv2.VideoCapture(0)  # 0 is typically the default camera

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture image")
        break

    # Flip the frame horizontally to avoid mirroring
    frame = cv2.flip(frame, 1)

    # Perform face detection on the colored image (no need to convert to grayscale)
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangle around the faces and predict age
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face_rgb = cv2.resize(face, (224, 224))  # Resize to match the input size of the model
        
        # Convert image to tensor and apply normalization
        face_rgb = cv2.cvtColor(face_rgb, cv2.COLOR_BGR2RGB)
        face_tensor = transform(face_rgb)
        face_tensor = face_tensor.unsqueeze(0)  # Add batch dimension

        # Predict age
        with torch.no_grad():
            predicted_age = model(face_tensor).item()

        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Put the predicted age on the frame
        cv2.putText(frame, f'Age: {int(predicted_age)}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face and Age Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 32ms/step
